# Strax raw data check

In [1]:
import glob
from tqdm import tqdm
import os 
import os.path as osp

dirname = '/dali/lgrandi/tunnell/strax_data'

import strax
import straxen
st = strax.Context(
    storage=strax.DataDirectory(dirname),
    register=straxen.plugins.pax_interface.RecordsFromPax,
    **{**straxen.common_opts, 
       **dict(check_available=('raw_records',))})


This will run for some time for a large directory over a network filesystem. It will show a progress bar, but no progress until is is done.

In [2]:
# Simple check
ok_runs = st.select_runs(available='raw_records')['name'].values.tolist()
len(ok_runs)

Checking data availability: 100%|██████████| 1/1 [00:02<00:00,  2.95s/it]


1996

In [3]:
# Detailed check
rr_hash = st._key_for('0', 'raw_records').lineage_hash
all_dirs = set(glob.glob(dirname + '/*'))
checked_ok = set()

for q in tqdm(all_dirs):
    if not osp.isdir(q):
        continue
    try:
        _run_id, _data_type, _hash = st.storage[0]._parse_folder_name(q)
    except strax.storage.files.InvalidFolderNameFormat:
        print(f"{q}: Invalid or non-strax folder name")
        continue
     
    if _data_type != 'raw_records':
        print(f"{q}: Non-raw data type")
        continue

    if _hash != rr_hash:
        print(f"{q}: Out of date or invalid hash")
        continue
    
    try:
        md = st.get_metadata(_run_id, 'raw_records')
    except strax.DataCorrupted:
        print(f"{q}: Exception while loading metadata")
        continue

    if 'exception' in md:
        print(f"{q}: Metadata contains exception (crash during conversion)")
        continue
    if not 'writing_ended' in md:
        print(f"{q}: Metadata indicates conversion never finished.")
        continue
    
    if not 'chunks' in md:
        print(f"{q}: No chunks registered in metadata")
        continue
        
    if not 'lineage_hash' in md:
        print(f"{q}: Lineage hash missing from metadata")
        continue

    fn = osp.join(q, md['chunks'][0]['filename'])
    if not osp.exists(fn) or osp.getsize(fn) == 0:
        print(f"{q}: First chunk file is non-existent or empty")
        continue
        
    # If you want to be really really sure the data is ok, you can try to load the data
    # try:
    #     rr = st.get_array(_run_id, 'raw_records', seconds_range=(0, 0.1))
    # except Exception as e:
    #     print(f"{q}: Exception while actually loading data: {e}")
    #     continue
    
    checked_ok.add(q)
    
failing = all_dirs - checked_ok
print(f"Found {len(failing)} bad and {len(checked_ok)} good data folders out of {len(all_dirs)}")

  6%|▌         | 116/2071 [00:00<00:03, 578.06it/s]

/dali/lgrandi/tunnell/strax_data/180706_2252-raw_records-7k65yaooed: Metadata contains exception (crash during conversion)
/dali/lgrandi/tunnell/strax_data/180125_0346-raw_records-7k65yaooed: Metadata contains exception (crash during conversion)
/dali/lgrandi/tunnell/strax_data/180123_2308-raw_records-7k65yaooed: Metadata contains exception (crash during conversion)
/dali/lgrandi/tunnell/strax_data/180205_0331-raw_records-7k65yaooed: Metadata contains exception (crash during conversion)


 13%|█▎        | 261/2071 [00:00<00:02, 645.84it/s]

/dali/lgrandi/tunnell/strax_data/180202_0836-raw_records-7k65yaooed: Metadata contains exception (crash during conversion)
/dali/lgrandi/tunnell/strax_data/180127_1021-raw_records-7k65yaooed: Metadata contains exception (crash during conversion)
/dali/lgrandi/tunnell/strax_data/180205_2348-raw_records-7k65yaooed: Metadata contains exception (crash during conversion)
/dali/lgrandi/tunnell/strax_data/180127_0520-raw_records-7k65yaooed: Metadata contains exception (crash during conversion)
/dali/lgrandi/tunnell/strax_data/180404_0835-raw_records-7k65yaooed: Metadata contains exception (crash during conversion)
/dali/lgrandi/tunnell/strax_data/180706_1551-raw_records-7k65yaooed: Metadata contains exception (crash during conversion)
/dali/lgrandi/tunnell/strax_data/180203_2325-raw_records-7k65yaooed: Metadata contains exception (crash during conversion)


 22%|██▏       | 450/2071 [00:00<00:02, 630.50it/s]

...

/dali/lgrandi/tunnell/strax_data/180403_1233-raw_records-7k65yaooed: Metadata contains exception (crash during conversion)
/dali/lgrandi/tunnell/strax_data/180918_1607-raw_records-7k65yaooed: Metadata contains exception (crash during conversion)


 97%|█████████▋| 2003/2071 [00:03<00:00, 659.23it/s]

/dali/lgrandi/tunnell/strax_data/180129_0518-raw_records-7k65yaooed: Metadata contains exception (crash during conversion)
/dali/lgrandi/tunnell/strax_data/180130_0622-raw_records-7k65yaooed: Metadata contains exception (crash during conversion)
/dali/lgrandi/tunnell/strax_data/180125_0546-raw_records-7k65yaooed: Metadata contains exception (crash during conversion)
/dali/lgrandi/tunnell/strax_data/180128_1426-raw_records-7k65yaooed: Metadata contains exception (crash during conversion)
/dali/lgrandi/tunnell/strax_data/180131_2230-raw_records-7k65yaooed: Metadata contains exception (crash during conversion)
/dali/lgrandi/tunnell/strax_data/180201_1332-raw_records-7k65yaooed: Metadata contains exception (crash during conversion)
/dali/lgrandi/tunnell/strax_data/180203_1925-raw_records-7k65yaooed: Metadata contains exception (crash during conversion)
/dali/lgrandi/tunnell/strax_data/180123_2208-raw_records-7k65yaooed: Metadata contains exception (crash during conversion)


100%|██████████| 2071/2071 [00:03<00:00, 659.80it/s]

/dali/lgrandi/tunnell/strax_data/180126_0651-raw_records-7k65yaooed: Metadata contains exception (crash during conversion)
/dali/lgrandi/tunnell/strax_data/180215_1035-raw_records-7k65yaooed: Metadata contains exception (crash during conversion)
Found 75 bad and 1996 good data folders out of 2071
